# SVAMITVA — DGX Training Pipeline

**Target:** DGX Server with Multiple GPUs (CUDA).
**DATA path:** `/jupyter/sods.user04/DATA`

Automatically discovers `MAP1`, `MAP2`, etc., and trains sequentially inheriting checkpoints (KMeans tile filtering enabled to avoid NoData).

---
## Cell 1 — Setup

In [ ]:
import os, sys, time, torch
from pathlib import Path
import logging
logging.getLogger().setLevel(logging.WARNING)
# =============================================================================== 
# 🛰️ DGX STABILITY FIX (MULTI-GPU, >40GB FREE)
# =============================================================================== 
def get_free_gpus(min_free_gb=40):
    import subprocess
    result = subprocess.run(
        ['nvidia-smi', '--query-gpu=memory.free', '--format=csv,nounits,noheader'],
        stdout=subprocess.PIPE, encoding='utf-8'
    )
    free_memories = [int(x) for x in result.stdout.strip().split('\n')]
    eligible_gpus = [str(i) for i, mem in enumerate(free_memories) if mem > min_free_gb * 1024]
    return eligible_gpus

gpu_ids = get_free_gpus()
if not gpu_ids:
    raise RuntimeError("No GPU with >40GB free found.")
os.environ["CUDA_VISIBLE_DEVICES"] = ",".join(gpu_ids)
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# --- 🧬 ROOT DISCOVERY ---
PROJECT_ROOT = Path.cwd()
for parent in [PROJECT_ROOT] + list(PROJECT_ROOT.parents):
    if (parent / "models").exists() or (parent / "requirements.txt").exists():
        PROJECT_ROOT = parent; break
else:
    PROJECT_ROOT = Path("/svamitva_model") if Path("/svamitva_model").exists() else Path.cwd()
if str(PROJECT_ROOT) not in sys.path: sys.path.insert(0, str(PROJECT_ROOT))

# --- 📁 DIRECTORY SETUP ---
DATA_DIR = Path("/jupyter/sods.user04/DATA")
if not DATA_DIR.exists(): DATA_DIR = Path("/DATA")
CKPT_DIR = Path("/jupyter/sods.user04/check")
CKPT_DIR.mkdir(parents=True, exist_ok=True)
LOG_DIR  = PROJECT_ROOT / "logs"; LOG_DIR.mkdir(exist_ok=True)

# --- ⚙️ DEVICE & CONFIG ---
device = torch.device("cuda")
CONFIG = dict(
    backbone="resnet50", pretrained=True, image_size=512, batch_size=32,
    epochs_per_map=50, learning_rate=2e-4, weight_decay=1e-4, num_workers=8,
    mixed_precision=True, gradient_clip=1.0,
    building_weight=1.0, roof_weight=0.5, road_weight=0.8,
    waterbody_weight=0.8, road_centerline_weight=0.7,
    waterbody_line_weight=0.7, waterbody_point_weight=0.9,
    utility_line_weight=0.7, utility_poly_weight=0.8,
    bridge_weight=1.0, railway_weight=0.9,
)

TARGET_KEYS = ["building_mask", "road_mask", "road_centerline_mask", "waterbody_mask", 
               "waterbody_line_mask", "waterbody_point_mask", "utility_line_mask", 
               "utility_poly_mask", "bridge_mask", "railway_mask", "roof_type_mask"]

print(f"✅ STABILITY MODE: Using GPUs {gpu_ids} (each >40GB free).")
print(f"✅ Setup Complete | Devices: {[torch.cuda.get_device_name(i) for i in range(torch.cuda.device_count())]}")

# When creating your model:
# model = ... # your model definition
# model = torch.nn.DataParallel(model)  # for multi-GPU training

---
## Cell 2 — Training Engine

In [ ]:
import sys, os, time, torch, torch.nn as nn
from pathlib import Path

# --- 🛡️ SELF-HEALING PATH CHECK (Crucial for imports) ---
if 'models' not in sys.modules:
    for _p in [Path.cwd()] + list(Path.cwd().parents):
        if (_p / "models").exists() and (_p / "models/__init__.py").exists():
            if str(_p) not in sys.path: sys.path.insert(0, str(_p))
            break
    else:
        dgx_path = "/jupyter/sods.user04/svamitva_model"
        if Path(dgx_path).exists() and dgx_path not in sys.path: 
            sys.path.insert(0, dgx_path)

# --- 🛰️ GPU SELECTION: Use all GPUs with >40GB free ---
def get_free_gpus(min_free_gb=40):
    import subprocess
    result = subprocess.run(
        ['nvidia-smi', '--query-gpu=memory.free', '--format=csv,nounits,noheader'],
        stdout=subprocess.PIPE, encoding='utf-8'
    )
    free_memories = [int(x) for x in result.stdout.strip().split('\n')]
    eligible_gpus = [str(i) for i, mem in enumerate(free_memories) if mem > min_free_gb * 1024]
    return eligible_gpus

gpu_ids = get_free_gpus()
if not gpu_ids:
    raise RuntimeError("No GPU with >40GB free found.")
os.environ["CUDA_VISIBLE_DEVICES"] = ",".join(gpu_ids)
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

device = torch.device("cuda")
torch.backends.cudnn.benchmark = True  # <--- Enable cuDNN autotune

# --- 🧬 ROOT DISCOVERY ---
PROJECT_ROOT = Path.cwd()
for parent in [PROJECT_ROOT] + list(PROJECT_ROOT.parents):
    if (parent / "models").exists() or (parent / "requirements.txt").exists():
        PROJECT_ROOT = parent; break
else:
    PROJECT_ROOT = Path("/svamitva_model") if Path("/svamitva_model").exists() else Path.cwd()

if str(PROJECT_ROOT) not in sys.path: sys.path.insert(0, str(PROJECT_ROOT))

# --- 📁 DIRECTORY SETUP ---
DATA_DIR = Path("/jupyter/sods.user04/DATA")
if not DATA_DIR.exists(): DATA_DIR = Path("/DATA")
CKPT_DIR = Path("/jupyter/sods.user04/check")  # Ensure this is the correct, writable path
CKPT_DIR.mkdir(parents=True, exist_ok=True)
LOG_DIR  = PROJECT_ROOT / "logs"; LOG_DIR.mkdir(exist_ok=True)

# --- ⚙️ DEVICE & CONFIG ---
CONFIG = dict(
    backbone="resnet50", pretrained=True, image_size=512, batch_size=32,
    epochs_per_map=50, learning_rate=2e-4, weight_decay=1e-4, num_workers=8,
    mixed_precision=True, gradient_clip=1.0,
    building_weight=1.0, roof_weight=0.5, road_weight=0.8,
    waterbody_weight=0.8, road_centerline_weight=0.7,
    waterbody_line_weight=0.7, waterbody_point_weight=0.9,
    utility_line_weight=0.7, utility_poly_weight=0.8,
    bridge_weight=1.0, railway_weight=0.9,
)

TARGET_KEYS = ["building_mask", "road_mask", "road_centerline_mask", "waterbody_mask", 
               "waterbody_line_mask", "waterbody_point_mask", "utility_line_mask", 
               "utility_poly_mask", "bridge_mask", "railway_mask", "roof_type_mask"]

# --- 📦 IMPORTS ---
from torch.utils.data import DataLoader
try: from torch.amp import GradScaler, autocast
except: from torch.cuda.amp import GradScaler, autocast

from models.feature_extractor import FeatureExtractor
from models.losses import MultiTaskLoss
from training.metrics import MetricTracker
from data.dataset import SvamitvaDataset
from data.augmentation import get_train_transforms

def move_targets(batch):
    return {k: v.to(device) for k, v in batch.items() if k in TARGET_KEYS}

def build_model(load_from: Path = None):
    m = FeatureExtractor(backbone=CONFIG["backbone"], pretrained=True, num_roof_classes=5)
    if load_from and load_from.exists():
        state = torch.load(load_from, map_location="cpu", weights_only=False)
        weights = state.get("model") or state.get("model_state_dict") or state
        m.load_state_dict(weights, strict=False)
    # Wrap model for multi-GPU
    if torch.cuda.device_count() > 1:
        m = nn.DataParallel(m)
    return m.to(device)

def train_map(map_name: str, resume_from: Path = None):
    torch.cuda.empty_cache()
    map_dir = DATA_DIR / map_name
    best_out, last_out = CKPT_DIR / f"{map_name}_best.pt", CKPT_DIR / f"{map_name}_latest.pt"
    if not map_dir.exists(): return best_out if best_out.exists() else None

    print(f"\n{'='*70}\n  Region     : {map_name}\n  Isolation  : GPUs {gpu_ids}\n{'='*70}")
    model_w = build_model(load_from=resume_from)
    
    try:
        ds = SvamitvaDataset(root_dir=DATA_DIR, image_size=512, transform=get_train_transforms(512), mode="train")
        ds.samples = [s for s in ds.samples if s["map_name"] == map_name]
        loader = DataLoader(ds, batch_size=CONFIG["batch_size"], shuffle=True, num_workers=CONFIG["num_workers"], pin_memory=True)
    except Exception as e: print(f"Data Fail: {e}"); return None

    loss_fn = MultiTaskLoss(**{k: v for k, v in CONFIG.items() if k.endswith("_weight")}).to(device)
    optimizer = torch.optim.AdamW(model_w.parameters(), lr=CONFIG["learning_rate"], weight_decay=CONFIG["weight_decay"])
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=1e-6)
    scaler = GradScaler(enabled=True)

    best_iou = 0.0
    try:
        for epoch in range(1, 51):
            model_w.train()
            tracker, run_loss, n_steps, t0 = MetricTracker(), 0.0, 0, time.time()
            for batch in loader:
                imgs, targets = batch["image"].to(device, non_blocking=True), move_targets(batch)
                optimizer.zero_grad(set_to_none=True)
                with autocast(device_type="cuda", enabled=True):
                    preds = model_w(imgs); total_loss, _ = loss_fn(preds, targets)
                if not torch.isfinite(total_loss): continue
                scaler.scale(total_loss).backward(); scaler.step(optimizer); scaler.update()
                run_loss += total_loss.item(); tracker.update(preds, targets); n_steps += 1
            
            scheduler.step()
            avg_iou = tracker.compute().get("avg_iou", 0.0)
            print(f"  Epoch {epoch:2d} | loss: {run_loss/max(n_steps,1):.4f} | iou: {avg_iou:.4f} | {time.time()-t0:.0f}s")
            print(f"Saving checkpoint to: {last_out}")
            torch.save(model_w.state_dict(), last_out)
            print(f"Checkpoint saved: {last_out}")  # <-- Live update print
            if avg_iou > best_iou:
                best_iou = avg_iou
                print(f"Saving best checkpoint to: {best_out}")
                torch.save(model_w.state_dict(), best_out)
                print(f"Best checkpoint saved: {best_out}")  # <-- Live update print
    except (Exception, KeyboardInterrupt) as e:
        print(f"\n⚠️ EMERGENCY SAVE: {e}")
        torch.save(model_w.state_dict(), last_out.with_suffix(".crash_backup.pt"))
        print(f"Crash backup saved: {last_out.with_suffix('.crash_backup.pt')}")
        raise e
    return best_out

print(f"✅ Logic Ready & Modules Imported | Using GPUs: {gpu_ids}")

---
## Cell 3 — Execute Training
Sequential multi-map training. Iterates through all available DGX MAP folders.

In [ ]:
import io
import sys
import re

def auto_train_all():
    # Discover MAP folders
    map_folders = sorted([
        d.name for d in DATA_DIR.iterdir()
        if d.is_dir() and d.name.startswith("MAP") and not d.name.startswith(".")
    ])
    print(f"🚀 Training {len(map_folders)} maps sequentially: {map_folders}")

    prev_ckpt, summary = None, []
    for m_name in map_folders:
        print(f"⏳ Training {m_name}... ", end="", flush=True)
        buffer = io.StringIO()
        old_out, old_err = sys.stdout, sys.stderr
        sys.stdout = sys.stderr = buffer
        err_msg = None
        try:
            ckpt = train_map(m_name, resume_from=prev_ckpt)
        except Exception as e:
            ckpt = None
            err_msg = str(e)
        finally:
            sys.stdout = old_out
            sys.stderr = old_err

        logs = buffer.getvalue().splitlines()
        # Preprocessing progress: only show total/completed pixels and percentage
        total_pixels = None
        completed_pixels = 0
        for line in logs:
            m = re.search(r"tiles from (MAP\d+) \((\d+)×(\d+)px\)", line)
            if m:
                total_pixels = int(m.group(2)) * int(m.group(3))
            m2 = re.search(r"mask positive pixels: (\d+)", line)
            if m2 and total_pixels:
                completed_pixels += int(m2.group(1))
                percent = int(completed_pixels / max(total_pixels, 1) * 100)
                print(f"   Preprocessing: {completed_pixels}/{total_pixels} px ({percent}%)")

        # Show percentage progress for each epoch
        epoch_lines = [l for l in logs if "Epoch" in l and "/" in l]
        for line in epoch_lines:
            m = re.search(r"Epoch (\d+)/(\d+)", line)
            if m:
                cur, total = int(m.group(1)), int(m.group(2))
                percent = int(cur / max(total, 1) * 100)
                print(f"   Progress: Epoch {cur}/{total} ({percent}%)")

        # Show checkpoint saving/updating
        for line in logs:
            if "checkpoint saved" in line or "Saving emergency checkpoint" in line or "New best!" in line:
                print(f"   {line}")

        # After training, print only the final result for this map
        if ckpt and ckpt.exists():
            print("✅ SUCCESS")
            summary.append(f"{m_name}: OK")
            prev_ckpt = ckpt
        else:
            print("❌ FAILED")
            summary.append(f"{m_name}: FAIL")
            if err_msg:
                print(f"   Reason: {err_msg}")
            error_lines = [l for l in logs if "ERROR" in l or "Exception" in l or "FAILED" in l]
            if error_lines:
                print("   Error log:")
                for l in error_lines[-5:]:
                    print(f"   {l}")
            break

    print("\n   *** SEQUENTIAL DGX TRAINING COMPLETE ***")
    for s in summary:
        print(f" - {s}")

auto_train_all()